#### 单线程

In [51]:
zl_stg=pd.read_csv(r'C:\TM1py\ZL_Amazon_Payments_Data_STG.csv',iterator=True,chunksize=10000,low_memory=False,
                   dtype={'SA Year':'str','Order Year':'str','COA Company':'str','Platform':'str'})

In [ ]:
from TM1py.Services import TM1Service
cubename='Bkp_ZL Amazon Payments Data STG'
for chunk in zl_stg:
    chunk.drop(['Cube'],axis=1,inplace=True)
    with TM1Service(address="192.168.1.176", port=30015, ssl=False, user="neil", password="123") as tm1:
        data=chunk
        tm1.cells.write_dataframe(cube_name=cubename,
                                       data=data,
                                       dimensions=['Scenario','SA Year','SA Month','Version','Order Year','Month Day','COA Company','Currency',
                                                   'Country','Region State','Region City','Platform','Behavior','Product','Data Source Payments Data STG',
                                                  'M ZL Amazon Payments Data STG'],
                                       increment=True,
                                       deactivate_transaction_log=True,
                                       reactivate_transaction_log=True,
                                      
                                       )

#### 多线程

In [ ]:
import pandas as pd
import asyncio
import nest_asyncio
nest_asyncio.apply()
from concurrent.futures import ThreadPoolExecutor
#from typing import List
from TM1py.Services import TM1Service

In [36]:
async def transfer_async(chunks:'chunks',tm1:TM1Service,cube_tgr:str):
    loop = asyncio.get_event_loop()
    
    with ThreadPoolExecutor(10) as executor:
        #futures = [loop.rudata:'pd.DataFrame',n_in_executor(executor, transfer, tm1_src, tm1_tgr,cube_src, cube_tgr,month_leaf) for month_leaf in leaves]
        futures=[]
        for chunk in chunks:
            chunk.drop(['Cube'],axis=1,inplace=True)
            f=loop.run_in_executor(executor, transfer,chunk,tm1,cube_tgr)
            futures.append(f)
        for future in futures:
            try:
                await future
            except:
                print(future)
                continue



def transfer(chunk:'pd.DataFrame',tm1:TM1Service,cube_tgr:str):
    #Clear data

    #Writing to tgr Cube
    tm1.cells.write_dataframe(
        cube_name=cube_tgr,
        data=chunk,
        dimensions=['Scenario','SA Year','SA Month','Version','Order Year','Month Day','COA Company','Currency',
                    'Country','Region State','Region City','Platform','Behavior','Product','Data Source Payments Data STG',
                    'M ZL Amazon Payments Data STG'],
        increment=False,
        deactivate_transaction_log=True,
        reactivate_transaction_log=True,
        sandbox_name=None,
        use_ti=True,
        skip_non_updateable=True,
        use_changeset=False)

In [52]:
with TM1Service(address="192.168.1.176", port=30015, ssl=False, user="neil", password="123") as tm1:
    #leaves=['Jul','Aug','Sep','Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun']
    asyncio.run(transfer_async(zl_stg,tm1,'Bkp_ZL Amazon Payments Data STG'))

ParserError: Error tokenizing data. C error: out of memory